<a href="https://colab.research.google.com/github/redmicelles/Mobalysis/blob/main/Classwork_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Customers.csv")
df.head()

,CustomerID,Gender,Age,Annual Income ($),Spending Score (1-100),Profession,Work Experience,Family Size
0,1,Male,19,15000,39,Healthcare,1,4
1,2,Male,21,35000,81,Engineer,3,3
2,3,Female,20,86000,6,Engineer,1,1
3,4,Female,23,59000,77,Lawyer,0,2
4,5,Female,31,38000,40,Entertainment,2,6


In [ ]:
df.describe()

,CustomerID,Age,Annual Income ($),Spending Score (1-100),Work Experience,Family Size
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,1000.500000,48.960000,110731.821500,50.962500,4.102500,3.768500
std,577.494589,28.429747,45739.536688,27.934661,3.922204,1.970749
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,500.750000,25.000000,74572.000000,28.000000,1.000000,2.000000
50%,1000.500000,48.000000,110045.000000,50.000000,3.000000,4.000000
75%,1500.250000,73.000000,149092.750000,75.000000,7.000000,5.000000
max,2000.000000,99.000000,189974.000000,100.000000,17.000000,9.000000


In [ ]:
null_counts = {column: df[column].isna().sum() for column in df.columns}
null_counts

{'CustomerID': 0,
 'Gender': 0,
 'Age': 0,
 'Annual Income ($)': 0,
 'Spending Score (1-100)': 0,
 'Profession': 35,
 'Work Experience': 0,
 'Family Size': 0}

In [ ]:
#Unique value count for profession
df["Profession"].nunique()

9

In [ ]:
#Professions
df["Profession"].value_counts()

Artist           612
Healthcare       339
Entertainment    234
Engineer         179
Doctor           161
Executive        153
Lawyer           142
Marketing         85
Homemaker         60
Name: Profession, dtype: int64

In [ ]:
df["Profession_filled"] = df["Profession"].fillna(df["Profession"].mode()[0])
null_counts = {column: df[column].isna().sum() for column in df.columns}
null_counts

{'CustomerID': 0,
 'Gender': 0,
 'Age': 0,
 'Annual Income ($)': 0,
 'Spending Score (1-100)': 0,
 'Profession': 35,
 'Work Experience': 0,
 'Family Size': 0,
 'Profession_filled': 0}

In [ ]:
df.dtypes

CustomerID                 int64
Gender                    object
Age                        int64
Annual Income ($)          int64
Spending Score (1-100)     int64
Profession                object
Work Experience            int64
Family Size                int64
Profession_filled         object
dtype: object

#**EDA**

---



In [ ]:
df_grouped = df.groupby("Gender")["Annual Income ($)"].mean()
df_grouped

Gender
Female    110553.715008
Male      110991.323096
Name: Annual Income ($), dtype: float64

In [ ]:
df.groupby("Profession_filled")["Annual Income ($)"].mean().sort_values(ascending=False).head(1)

Profession_filled
Executive    113770.130719
Name: Annual Income ($), dtype: float64

In [ ]:
df.sort_values("Annual Income ($)", ascending=False).iloc[0]

CustomerID                     570
Gender                      Female
Age                             91
Annual Income ($)           189974
Spending Score (1-100)          37
Profession                Engineer
Work Experience                  8
Family Size                      1
Profession_filled         Engineer
Name: 569, dtype: object

In [ ]:
df.groupby(["Profession_filled", "Gender"])["Gender"].count().head(4)

Profession_filled  Gender
Artist             Female    400
                   Male      247
Doctor             Female     89
                   Male       72
Name: Gender, dtype: int64

In [ ]:
#Label Encoding
from sklearn.preprocessing  import LabelEncoder
transformer = LabelEncoder()
df['Gender_Encoded'] = transformer.fit_transform(df['Gender'])
df.head()

,CustomerID,Gender,Age,Annual Income ($),Spending Score (1-100),Profession,Work Experience,Family Size,Profession_filled,Gender_Encoded
0,1,Male,19,15000,39,Healthcare,1,4,Healthcare,1
1,2,Male,21,35000,81,Engineer,3,3,Engineer,1
2,3,Female,20,86000,6,Engineer,1,1,Engineer,0
3,4,Female,23,59000,77,Lawyer,0,2,Lawyer,0
4,5,Female,31,38000,40,Entertainment,2,6,Entertainment,0


In [86]:
# Dictionary Encoding
groupings = df.groupby("Profession_filled")["Annual Income ($)"].agg(avg_income="mean").sort_values("avg_income", ascending=False)
print(groupings)
# groupings_dict = {groupings.iloc[x].name: x for x in range(len(groupings))}
groupings_dict = {}
for i in range(len(groupings)):
  groupings_dict[groupings.iloc[i].name] = i
print(groupings_dict)

                      avg_income
Profession_filled               
Executive          113770.130719
Healthcare         112574.041298
Doctor             111573.217391
Engineer           111161.240223
Lawyer             110995.838028
Entertainment      110650.333333
Artist             109234.081917
Homemaker          108758.616667
Marketing          107994.211765
{'Executive': 0, 'Healthcare': 1, 'Doctor': 2, 'Engineer': 3, 'Lawyer': 4, 'Entertainment': 5, 'Artist': 6, 'Homemaker': 7, 'Marketing': 8}


In [ ]:
df['Profession_Ordinal'] = df['Profession_filled'].replace(groupings_dict)

df.head(20)

,CustomerID,Gender,Age,Annual Income ($),Spending Score (1-100),Profession,Work Experience,Family Size,Profession_filled,Gender_Encoded,Profession_Ordinal
0,1,Male,19,15000,39,Healthcare,1,4,Healthcare,1,1
1,2,Male,21,35000,81,Engineer,3,3,Engineer,1,3
2,3,Female,20,86000,6,Engineer,1,1,Engineer,0,3
3,4,Female,23,59000,77,Lawyer,0,2,Lawyer,0,4
4,5,Female,31,38000,40,Entertainment,2,6,Entertainment,0,5
5,6,Female,22,58000,76,Artist,0,2,Artist,0,6
6,7,Female,35,31000,6,Healthcare,1,3,Healthcare,0,1
7,8,Female,23,84000,94,Healthcare,1,3,Healthcare,0,1
8,9,Male,64,97000,3,Engineer,0,3,Engineer,1,3
9,10,Female,30,98000,72,Artist,1,4,Artist,0,6


In [89]:
#!pip install pandas_profilling

{'Executive': 0, 'Healthcare': 1, 'Doctor': 2}